In [1]:
#AUTHORIZE

#https://medium.com/swlh/using-python-to-connect-to-stravas-api-and-analyse-your-activities-dummies-guide-5f49727aac86
#http://www.strava.com/oauth/authorize?client_id=01234&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=read,activity:read_all,activity:write

import requests
import json
# Make Strava auth API call with your 
# client_code, client_secret and code
response = requests.post(
                    url = 'https://www.strava.com/oauth/token',
                    data = {
                            'client_id': 01234,
                            'client_secret': '---',
                            'code': '---',
                            'grant_type': 'authorization_code'
                            }
                )
#Save json response as a variable
strava_tokens = response.json()
# Save tokens to file
with open('strava_tokens.json', 'w') as outfile:
    json.dump(strava_tokens, outfile)
# Open JSON file and print the file contents 
# to check it's worked properly
with open('strava_tokens.json') as check:
  data = json.load(check)
print(data)

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-1-bf2c305138b0>, line 13)

In [31]:
#refresh token
def refresh_token():
    import requests
    import json
    import time

    # Get the tokens from file to connect to Strava
    with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)
    # If access_token has expired then 
    # use the refresh_token to get the new access_token
    if strava_tokens['expires_at'] < time.time():
    # Make Strava auth API call with current refresh token
        print('token refreshed')
        response = requests.post(
                            url = 'https://www.strava.com/oauth/token',
                            data = {
                                    'client_id': 01234,
                                    'client_secret': '---',
                                    'grant_type': 'refresh_token',
                                    'refresh_token': strava_tokens['refresh_token']
                                    }
                        )
    # Save response as json in new variable
        new_strava_tokens = response.json()
    # Save new tokens to file
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
    # Use new Strava tokens from now
        strava_tokens = new_strava_tokens
    # Open the new JSON file and print the file contents 
    # to check it's worked properly
    with open('strava_tokens.json') as check:
      data = json.load(check)
    print(data)

In [3]:
def get_activitydata(idpar):
    url = "https://www.strava.com/api/v3/activities/" + str(idpar) + "/streams/time"
    r = requests.get(url + '?access_token=' + access_token + '&types=["time"]&key_by_type=true')
    r = r.json()
    return r

In [4]:
def create_dataframe(r):
    df = pd.DataFrame(r['distance']['data'])
    df2 = pd.DataFrame(r['time']['data'])
    df3= pd.concat([df, df2], axis=1)
    df_res = pd.concat([df3.shift(1), df3], axis=1)
    df_res.columns = ['dist-1', 'time-1', 'dist', 'time']
    df_res['time_diff'] = df_res['time'] - df_res['time-1']
    df_res['dist_diff'] = df_res['dist'] - df_res['dist-1']
    df_res['speed'] = (df_res['dist_diff'] / df_res['time_diff']) * 3.6
    df_res['sumcumtimediff'] =  df_res.sort_values(by=['speed'], ascending=True)['time_diff'].cumsum()
    df_res['sumcumdistdiff'] =  df_res.sort_values(by=['speed'], ascending=True)['dist_diff'].cumsum()
    time = df_res.loc[df_res['speed'].size-1]['time']
    df_res['percentile'] = (df_res['sumcumtimediff']) / time 
    df_res['percentilerounded'] = round((df_res['percentile'] * 100) , 0) 
    return df_res

In [5]:
def calculate_split(df_res, minutes):
    lst = []
    #minutes = 20
    interval = minutes * 60

    for index, row in df_res.iterrows():    
        if df_res[df_res['time']<=row['time']-interval]['time'].size != 0:
            val = max(df_res[df_res['time']<=row['time']-interval]['time'])
            record = df_res[df_res['time']==val]
            timediff = row['time'] - record['time']
            distdiff = row['dist'] - record['dist']
            speed = (distdiff / timediff) * 3.6
            #print(str(row['time']) + " avg speed " + str(speed.to_string(index=False)))
            lst.append(float(speed.to_string(index=False)))
        else:
            lst.append(0)
    
    
    val = max(lst)
    p = lst.index(val)
    df2 = df_res.filter(items = [p], axis=0)
    
    timeval = max(df_res[df_res['time']<=int(float((df2['time']-(60*minutes)).to_string(index=False)))]['time'])
    kmrecord = df_res[df_res['time']==timeval]
    #print(int(float((df2['time']-(60*minutes)).to_string(index=False))))
    
    
    return [str(round(val,2)), str(int(float(((df2['time']/60)-minutes).to_string(index=False)))),
           str(round(float(((kmrecord['dist']/1000)).to_string(index=False)),2))]

In [6]:
def update_post(idpar, datapar):
    endpoint = "https://www.strava.com/api/v3/activities/" + str(idpar)
    data = datapar
    headers = {"Authorization": "Bearer " + access_token}

    requests.put(endpoint, data=data, headers=headers).json()
    print('updated')

In [32]:
def update_activity_post(idpar, distpar, timepar):
    r = get_activitydata(idpar)
    
    if list(r.keys())[0] == 'distance':
        df_res = create_dataframe(r)

        if max(df_res['time']) / 60 > 10:
            result_10 = calculate_split(df_res, 10)
        else:
            result_10 = ['0', '0', '0']
        if max(df_res['time']) / 60 > 20:
            result_20 = calculate_split(df_res, 20)
        else:
            result_20 = ['0', '0', '0']
        if max(df_res['time']) / 60 > 30:
            result_30 = calculate_split(df_res, 30)
        else:
            result_30 = ['0', '0', '0']

        subset = df_res[df_res['speed']<=20]

        maxtimediff = max(subset['sumcumtimediff'])

        maxdistdiff = max(subset['sumcumdistdiff']) 
        
        movingspeed = round((((distpar - maxdistdiff) / (timepar - maxtimediff)) * 3.6),2)
        
        timemoving = round(((timepar - maxtimediff) / 60),0)
        
        prctimemoving = round((((timepar - maxtimediff) / timepar) * 100),0)
        
        data = {
            'description': 'Avg. Moving speed (20+): ' + str(movingspeed) + ' (' + str(prctimemoving) + '%, ' + str(timemoving) + ' mins )' + '''
50% Qrt Speed: ''' + str(round(min(df_res[df_res['percentilerounded']==50]['speed']),2)) + '''
Best 10 min Speed: ''' + result_10[0] + ' @ ' + result_10[1] + ' min & ' + result_10[2] + ' km.' + '''
Best 20 min Speed: ''' + result_20[0] + ' @ ' + result_20[1] + ' min & ' + result_20[2] + ' km.' + '''
Best 30 min Speed: ''' + result_30[0] + ' @ ' + result_30[1] + ' min & ' + result_30[2] + ' km.' + '''
75% Qrt Speed: ''' + str(round(min(df_res[df_res['percentilerounded']==75]['speed']),2))
                }

        update_post(idpar, data)
    

In [2]:
#GET ACTIVITIES

import requests
import pandas as pd
from pandas.io.json import json_normalize
import json
import csv
refresh_token()
# Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
# Loop through all activities
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']
# Get first page of activities from Strava with all fields
r = requests.get(url + '?access_token=' + access_token)
r = r.json()
    
df = pd.json_normalize(r)
#df.to_csv('strava_activities_all_fields.csv')
df.head(n=20)

for index, acrow in df.head(n=2).iterrows():
    print(acrow['id'])
    if acrow['distance'] > 2000 and acrow['type'] == 'Ride':  
        update_activity_post(acrow['id'], acrow['distance'], acrow['elapsed_time'] )

NameError: name 'refresh_token' is not defined

In [57]:
#Testing

In [18]:
import pandas as pd
access_token = strava_tokens['access_token']
test = create_dataframe(get_activitydata(7003567797))


,dist-1,time-1,dist,time,time_diff,dist_diff,speed,sumcumtimediff,sumcumdistdiff,percentile,percentilerounded
627,7368.0,994.0,7368.0,995,1.0,0.0,0.00,1.0,0.0,0.000418,0.0
640,7375.1,1015.0,7375.1,1016,1.0,0.0,0.00,2.0,0.0,0.000836,0.0
639,7375.1,1013.0,7375.1,1015,2.0,0.0,0.00,4.0,0.0,0.001672,0.0
637,7371.6,1010.0,7371.6,1012,2.0,0.0,0.00,6.0,0.0,0.002508,0.0
636,7371.6,1009.0,7371.6,1010,1.0,0.0,0.00,7.0,0.0,0.002926,0.0
...,...,...,...,...,...,...,...,...,...,...,...
789,8995.5,1252.0,8998.6,1253,1.0,3.1,11.16,181.0,106.8,0.075669,8.0
1092,12545.8,1715.0,12552.1,1717,2.0,6.3,11.34,183.0,113.1,0.076505,8.0
1013,11778.4,1598.0,11784.8,1600,2.0,6.4,11.52,185.0,119.5,0.077341,8.0
17,88.9,21.0,92.1,22,1.0,3.2,11.52,186.0,122.7,0.077759,8.0


In [46]:
test.round(decimals = 2).to_csv('check.csv')